#### **Definindo a pipeline:**
**1. Definir o Problema de Negócio**

**2. Coletar os Dados**

**3. Descrição dos Dados**

**4. Análise Inicial dos Dados**
   - **4.1 Dimensões dos Dados**
    
   - **4.2 Tipos de Dados**
     
   - **4.3 Verificação de Valores Ausentes (NA)**
     
   - **4.4 Alterar Tipo dos Dados**
    
   - **4.5 Análise Estatística**
    
   - **4.6 Cadinalidade dos dados**

 
 **5. Feature Engineering**
   - **5.1 Tratando os valores Nulos**
    
   - **5.2 Engenharia de Novas Features**

**6. Análise Exploratória dos Dados (EDA)**
   
**7. Preparação dos Dados**
   - **7.1 Codificação (Encoder)**
   - **7.2 Separando o conjunto de dados**
   - **7.3 Seleção de Features** 
   - **7.4 Escalonamento**
     
**8. Machine Learning**

 **9. Avaliação de Desempenho do Modelo**
  


In [54]:
import pandas as pd
import numpy as np
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

Importanto os dados que passaram pela Feature Engineering

In [2]:
import pickle
with open ('train.pkl', 'rb') as f:
    train = pickle.load(f)

#### 7. Preparação dos Dados

Excluindo a coluna "Item_Identifier" pois não será util nas próxima etapas, e se permanecer irá atrapalhar o encoder.

In [3]:
train = train.drop(['Item_Identifier'], axis=1)

**7.1 Codificação (Encoder)**

Irei utilizar o LabelEncoder para transformar as colunas "Item_Fat_Content", "Outlet_Size", "Outlet_Location_Type", "Outlet_Type", "Item_Weight_Category" e "Item_MRP_Category" em numéricas e em seguida utilizarei o OneHotEncoder nas colunas "Item_Identifier", "Item_Type" e "Outlet_Identifier".

In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [5]:
label_encoders = {}
categorical_le = ['Item_Fat_Content', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type', 'Item_Weight_Category', 'Item_MRP_Category']
                                                                          
for column in categorical_le:
    le = LabelEncoder()
    train[column] = le.fit_transform(train[column])
    label_encoders[column] = le

In [6]:
oht_Item_Type = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
oht_Outlet_Identifier = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Coluna Item_Type
oht_Item_Type.fit(train[['Item_Type']])
oht_Item_Type_train = oht_Item_Type.transform(train[['Item_Type']])                                                               
oht_Item_Type_train_df = pd.DataFrame(oht_Item_Type_train, columns=oht_Item_Type.get_feature_names_out(['Item_Type']))

# Coluna Outlet_Identifier
oht_Outlet_Identifier.fit(train[['Outlet_Identifier']])
oht_Outlet_Identifier_train = oht_Outlet_Identifier.transform(train[['Outlet_Identifier']])                                                               
oht_Outlet_Identifier_train_df = pd.DataFrame(oht_Outlet_Identifier_train, columns=oht_Outlet_Identifier.get_feature_names_out(['Outlet_Identifier']))

# Removendo as colunas categóricas originais
train = train.drop(['Item_Type', 'Outlet_Identifier'], axis=1)

# Concatenando os dados codificados com as demais colunas                                                                         
train = pd.concat([train, oht_Item_Type_train_df, oht_Outlet_Identifier_train_df], axis=1)


**7.2 Separando o conjunto de dados**

Separandoa variável dependente e as variáveis independentes.

In [7]:
X = train.drop(['Item_Outlet_Sales'], axis=1)
y = train.Item_Outlet_Sales	

In [8]:
# Importanto Biblioteca para dividir o conjunto de dados.
from sklearn.model_selection import train_test_split

Utilizando o train_test_split para separar o conjunto de dados em train e test.

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Agora separando o conjunto de treino entre train e val(validação).

In [10]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

**7.3 Seleção de Features**

Eliminando algumas colunas com menores correlações com o nosso target para não atrapalhar o desempenho do modelo.

In [11]:
X_train = X_train.drop([ 'Item_Weight_Category', 'Item_Weight', 'Outlet_Establishment_Year',
                       ], axis=1)
X_test = X_test.drop(['Item_Weight_Category', 'Item_Weight', 'Outlet_Establishment_Year'], axis=1)
X_val = X_val.drop(['Item_Weight_Category', 'Item_Weight', 'Outlet_Establishment_Year'], axis=1)

**7.4 Escalonamento**

In [12]:
# Importando bibilioteca para realizar o escalonamento.
from sklearn.preprocessing import StandardScaler

In [13]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

#### 8. Machine Learning

In [30]:
# Importanto modelos.
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPRegressor

# Importando métricas de predição.
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error, r2_score

# Tunagem de hiperparâmetros
from sklearn.model_selection import GridSearchCV

# Biblioteca de visualização.
import plotly.graph_objects as go
import plotly.express as px


Testando os seguintes modelos para identificar qual tem um melhor desempenho para realizar a tunagem:
- Regressão Linear,
- Regressão Polinomial 
- Árvore de Decisão
- Random Forest
- SVM
- Redes Neurais


In [45]:
# Regressão Linear
linear_reg = LinearRegression()  
linear_reg.fit(X_train, y_train)
y_pred_linear = linear_reg.predict(X_val)
error_linear_regression = mean_absolute_error(y_val, y_pred_linear)

# Regressão Polinomial
poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train)
X_val_poly = poly.transform(X_val)
poly_reg = LinearRegression()  
poly_reg.fit(X_train_poly, y_train)
y_pred_poly = poly_reg.predict(X_val_poly)
error_poly = mean_absolute_error(y_val, y_pred_poly)

# Regressão com Árvore de Decisão
decision_tree_reg = DecisionTreeRegressor()
decision_tree_reg.fit(X_train, y_train)
y_pred_decision_tree = decision_tree_reg.predict(X_val)
error_decision_tree = mean_absolute_error(y_val, y_pred_decision_tree)

# Regressão com Random Forest
random_forest_reg = RandomForestRegressor(n_estimators=100)
random_forest_reg.fit(X_train, y_train)
y_pred_random_forest = random_forest_reg.predict(X_val)
error_random_forest = mean_absolute_error(y_val, y_pred_random_forest)

# Regressão com Vetores de Suporte
svm = SVR()
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_val)
error_svm = mean_absolute_error(y_val, y_pred_svm)

# Regressão com Redes Neurais
redes_neurais = MLPRegressor(max_iter=3000)
redes_neurais.fit(X_train, y_train)
y_pred_neurais = redes_neurais.predict(X_val)
error_redes_neurais = mean_absolute_error(y_val, y_pred_neurais)


In [46]:
error = pd.DataFrame({
    'Modelo': ['Regressão Linear', 'Regressão Polinomial',  'Árvore de Decisão', 'Random Forest', 'SVM', 'Redes Neurais'],
    'Erro': [error_linear_regression, error_poly, error_decision_tree, error_random_forest, error_svm, error_redes_neurais]
})
error  

,Modelo,Erro
0,Regressão Linear,877.586479
1,Regressão Polinomial,810.702066
2,Árvore de Decisão,1093.873279
3,Random Forest,832.500622
4,SVM,1282.435074
5,Redes Neurais,805.646806


Observamos um desempenho superior com o modelo de regressão baseado em redes neurais, pois conseguimos alcançar uma margem de erro menor em comparação com outros modelos. O próximo passo será otimizar ainda mais esse modelo para obter um desempenho ainda melhor. Para isso, utilizaremos o GridSearchCV, uma técnica que permite a busca por hiperparâmetros ótimos, ajustando o modelo para alcançar o melhor desempenho possível.

Parâmetros que serão testados:

In [19]:
mlp = MLPRegressor(max_iter=1000)
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],  
    'activation': ['relu', 'tanh', 'logistic'],  
    'solver': ['adam', 'lbfgs'],  
    'alpha': [0.0001, 0.001, 0.01],  
    'learning_rate': ['constant', 'invscaling', 'adaptive']  
}
grid_search = GridSearchCV(mlp, param_grid, cv=5, n_jobs=-1, verbose=2)

In [20]:
grid_search.fit(X_train, y_train)
print(f"Melhores parâmetros: {grid_search.best_params_}")
print(f"Melhor score: {grid_search.best_score_}")

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Melhores parâmetros: {'activation': 'relu', 'alpha': 0.001, 'hidden_layer_sizes': (50,), 'learning_rate': 'constant', 'solver': 'adam'}
Melhor score: 0.5896984634839415


C:\Users\Thais\anaconda3\Anaconda\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [21]:
test_score = grid_search.score(X_val, y_val)

y_val_pred = grid_search.predict(X_val)
val_mae = mean_absolute_error(y_val, y_val_pred)

print(f"Score no conjunto de validação: {test_score}")
print(f" Erro médio absoluto: {val_mae}")

Score no conjunto de validação: 0.5790773098316123
 Erro médio absoluto: 798.4050751893641


Conseguimos reduzir o erro médio de 805 para 798, o que representa uma melhoria significativa na precisão do modelo.

Aplicando os melhores parâmetro no conjunto de teste.

In [55]:
neurais_best_model = MLPRegressor(
                     max_iter=1000,
                     activation='relu',
                     alpha=0.0001, 
                     hidden_layer_sizes=(50,), 
                     learning_rate='invscaling', 
                     solver='adam'              
)
neurais_best_model.fit(X_train, y_train)
y_pred_best_model = neurais_best_model.predict(X_test)
error_best_model = mean_absolute_error(y_test, y_pred_best_model)


In [56]:
error_best_model

717.9639627451489

Conseguimos um desempenho ainda melhor, o erro médio absoluto igual a 717 no conjunto de teste.

#### 9. Avaliação de Desempenho do Modelo

Visualização do resultado obtido.

In [51]:
scatter_plot = px.scatter(x=y_test, y=y_pred_best_model, labels={
    "x": "Valores Reais (y_test)",
    "y": "Valores Preditos (y_pred_best_model)"
}, title="Valores Reais vs Preditos")

# Adicionar linha y=x para representar predições perfeitas
scatter_plot.add_trace(go.Scatter(x=y_test, y=y_test, mode='lines', name='y = x', line=dict(color='red', dash='dash')))

# Mostrar os gráficos
scatter_plot.show()


Após treinar o modelo para prever as vendas com base nos dados, apliquei uma série de métricas de avaliação para medir seu desempenho. As métricas utilizadas foram: 
- Erro Médio Absoluto (MAE) 
- Erro Quadrático Médio (MSE) 
- Raiz do Erro Quadrático Médio (RMSE) 
- R² (Coeficiente de Determinação). 

Essas métricas ajudam a avaliar tanto a precisão quanto a capacidade do modelo de explicar a variabilidade dos dados.

In [52]:
# Calcular MAE
mae = mean_absolute_error(y_test, y_pred_best_model)

# Calcular MSE 
mse = mean_squared_error(y_test, y_pred_best_model)

# Calcular RMSE
rmse = mse**0.5

# Calcular R²
r2 = r2_score(y_test, y_pred_best_model)


In [53]:
Desempenho = pd.DataFrame({
    'Métrica': ['Mean Absolute Error (MAE)','Mean Squared Error (MSE)','Root Mean Squared Error (RMSE)', 'R^2 Score'],     
    'Score': [round(mae, 2), round(mse, 2), round(rmse, 2), round(r2, 2)]
})
Desempenho

,Métrica,Score
0,Mean Absolute Error (MAE),715.60
1,Mean Squared Error (MSE),1039391.66
2,Root Mean Squared Error (RMSE),1019.51
3,R^2 Score,0.62


- MAE (Erro Médio Absoluto): O valor de 715.60 indica que, em média, o modelo erra a previsão das vendas em aproximadamente 714 unidades monetárias. Essa métrica nos dá uma ideia da magnitude dos erros.

- MSE (Erro Quadrático Médio): O MSE, que é uma medida mais sensível a grandes erros, foi de 1.0395.391,66. Valores maiores indicam que o modelo ainda comete alguns erros mais significativos.

- RMSE (Raiz do Erro Quadrático Médio): O RMSE é uma métrica que também leva em conta grandes erros, e neste caso foi de 1.019,61. Ele mostra uma estimativa da média de erro esperada para cada previsão de vendas.

- R² (Coeficiente de Determinação): O valor de 0.62 mostra que o modelo explica aproximadamente 62% da variação nas vendas. 